In [4]:
import re
import numpy as np
from typing import List, Dict, Tuple, Set
from dataclasses import dataclass
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
import json
from datetime import datetime
import warnings
from difflib import SequenceMatcher

# Suprime avisos específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

@dataclass
class AnaliseSemantica:
    """Armazena resultados da análise semântica"""
    similaridade: float
    trechos_similares: List[Tuple[str, str, float]]
    palavras_chave_comuns: List[str]
    contexto_similar: bool
    nivel_similaridade: str  # 'Alto', 'Médio', 'Baixo'

@dataclass
class AnaliseSintatica:
    """Armazena resultados da análise sintática"""
    similaridade: float
    padrao_estrutural: float
    sequencias_identicas: List[str]
    alteracoes_detectadas: List[Tuple[str, str]]
    nivel_similaridade: str  # 'Alto', 'Médio', 'Baixo'

@dataclass
class PlagiarismReport:
    """Classe para armazenar resultados detalhados da análise de plágio."""
    texto_original: str
    texto_suspeito: str
    analise_semantica: AnaliseSemantica
    analise_sintatica: AnaliseSintatica
    plagio_detectado: bool
    data_analise: str
    metadados: Dict

class EnhancedPlagioDetector:
    def __init__(self, 
                limiar_sintatico: float = 0.7,
                limiar_semantico: float = 0.7,
                tamanho_minimo_trecho: int = 10):
        """
        Inicializa o detector de plágio avançado.
        
        Args:
            limiar_sintatico: Limiar para similaridade sintática
            limiar_semantico: Limiar para similaridade semântica
            tamanho_minimo_trecho: Número mínimo de palavras para considerar um trecho
        """
        # Configuração do NLTK
        try:
            nltk.data.find('tokenizers/punkt')
            nltk.data.find('corpora/stopwords')
        except LookupError:
            print("Baixando recursos necessários do NLTK...")
            nltk.download('punkt', quiet=True)
            nltk.download('stopwords', quiet=True)

        self.vectorizador_tfidf = TfidfVectorizer(
            ngram_range=(1, 3),
            stop_words=set(stopwords.words('portuguese'))
        )
        self.limiar_sintatico = limiar_sintatico
        self.limiar_semantico = limiar_semantico
        self.tamanho_minimo_trecho = tamanho_minimo_trecho
        self.historico_analises = []
        self.stopwords = set(stopwords.words('portuguese'))

    def preprocessar_texto(self, texto: str) -> str:
        """
        Preprocessa o texto aplicando várias transformações.
        """
        if not isinstance(texto, str):
            raise ValueError("O texto deve ser uma string")
            
        # Remove caracteres especiais mantendo espaços
        texto = re.sub(r'[^\w\s]', '', texto)
        # Converte para minúsculas
        texto = texto.lower()
        # Remove espaços extras
        texto = ' '.join(texto.split())
        return texto

    def calcular_similaridade_sequencia(self, texto1: str, texto2: str) -> float:
        """
        Calcula a similaridade entre duas strings usando SequenceMatcher.
        """
        return SequenceMatcher(None, texto1, texto2).ratio()

    def calcular_similaridade_tfidf(self, texto1: str, texto2: str) -> float:
        """
        Calcula similaridade usando TF-IDF e similaridade do cosseno.
        """
        if not texto1 or not texto2:
            return 0.0
            
        textos = [texto1, texto2]
        try:
            tfidf_matriz = self.vectorizador_tfidf.fit_transform(textos)
            similaridade = cosine_similarity(tfidf_matriz[0:1], tfidf_matriz[1:2])
            return float(similaridade[0][0])
        except Exception as e:
            print(f"Erro ao calcular similaridade TF-IDF: {e}")
            return 0.0

    def dividir_em_trechos(self, texto: str) -> List[str]:
        """
        Divide o texto em trechos menores para análise detalhada.
        """
        try:
            sentencas = sent_tokenize(texto)
            trechos = []
            trecho_atual = ""
            
            for sentenca in sentencas:
                palavras_atual = len(trecho_atual.split())
                palavras_sentenca = len(sentenca.split())
                
                if palavras_atual + palavras_sentenca <= self.tamanho_minimo_trecho:
                    trecho_atual += " " + sentenca if trecho_atual else sentenca
                else:
                    if trecho_atual:
                        trechos.append(trecho_atual.strip())
                    trecho_atual = sentenca
                    
            if trecho_atual:
                trechos.append(trecho_atual.strip())
                
            return trechos
        except Exception as e:
            print(f"Erro ao dividir texto em trechos: {e}")
            return [texto]

    def encontrar_trechos_similares(self, texto1: str, texto2: str) -> List[Tuple[str, str, float]]:
        """
        Encontra pares de trechos similares entre os dois textos.
        """
        trechos1 = self.dividir_em_trechos(texto1)
        trechos2 = self.dividir_em_trechos(texto2)
        
        trechos_similares = []
        
        for t1 in trechos1:
            for t2 in trechos2:
                try:
                    similaridade = self.calcular_similaridade_sequencia(t1, t2)
                    if similaridade >= self.limiar_semantico:
                        trechos_similares.append((t1, t2, similaridade))
                except Exception as e:
                    print(f"Erro ao comparar trechos: {e}")
                    continue
                    
        return sorted(trechos_similares, key=lambda x: x[2], reverse=True)

    def encontrar_sequencias_identicas(self, texto1: str, texto2: str) -> List[str]:
        """
        Encontra sequências de palavras idênticas entre os textos.
        """
        words1 = texto1.split()
        words2 = texto2.split()
        sequencias = []
        
        for i in range(len(words1)):
            for j in range(len(words2)):
                k = 0
                while (i + k < len(words1) and j + k < len(words2) and 
                       words1[i + k] == words2[j + k]):
                    k += 1
                if k >= 3:  # Sequências de 3 ou mais palavras
                    sequencia = ' '.join(words1[i:i + k])
                    if not all(word in self.stopwords for word in sequencia.split()):
                        sequencias.append(sequencia)
        
        return list(set(sequencias))

    def detectar_alteracoes(self, texto1: str, texto2: str) -> List[Tuple[str, str]]:
        """
        Detecta alterações simples como substituições de palavras.
        """
        palavras1 = word_tokenize(texto1)
        palavras2 = word_tokenize(texto2)
        alteracoes = []
        
        for p1, p2 in zip(palavras1, palavras2):
            if p1 != p2 and len(p1) > 3 and len(p2) > 3:
                if p1 not in self.stopwords and p2 not in self.stopwords:
                    similaridade = self.calcular_similaridade_sequencia(p1, p2)
                    if similaridade > 0.5:  # Palavras similares
                        alteracoes.append((p1, p2))
        
        return alteracoes

    def calcular_padrao_estrutural(self, texto1: str, texto2: str) -> float:
        """
        Calcula similaridade estrutural entre os textos.
        """
        sent1 = sent_tokenize(texto1)
        sent2 = sent_tokenize(texto2)
        
        # Compara número de sentenças e comprimento médio
        len_ratio = min(len(sent1), len(sent2)) / max(len(sent1), len(sent2))
        
        avg_len1 = np.mean([len(s.split()) for s in sent1])
        avg_len2 = np.mean([len(s.split()) for s in sent2])
        
        len_similarity = 1 - abs(avg_len1 - avg_len2) / max(avg_len1, avg_len2)
        
        return (len_ratio + len_similarity) / 2

    def identificar_palavras_chave_comuns(self, texto1: str, texto2: str) -> List[str]:
        """
        Identifica palavras-chave importantes que aparecem em ambos os textos.
        """
        def get_keywords(texto: str) -> Set[str]:
            words = word_tokenize(texto.lower())
            return {w for w in words if w not in self.stopwords and len(w) > 3}
        
        keywords1 = get_keywords(texto1)
        keywords2 = get_keywords(texto2)
        
        return sorted(list(keywords1.intersection(keywords2)))

    def analisar_contexto(self, texto1: str, texto2: str) -> bool:
        """
        Analisa se os textos compartilham o mesmo contexto/tema.
        """
        keywords1 = set(self.identificar_palavras_chave_comuns(texto1, texto1))
        keywords2 = set(self.identificar_palavras_chave_comuns(texto2, texto2))
        
        overlap = len(keywords1.intersection(keywords2))
        total = len(keywords1.union(keywords2))
        
        return (overlap / total if total > 0 else 0) > 0.3

    def realizar_analise_sintatica(self, texto1: str, texto2: str) -> AnaliseSintatica:
        """
        Realiza análise sintática detalhada entre dois textos.
        """
        similaridade = self.calcular_similaridade_tfidf(texto1, texto2)
        sequencias = self.encontrar_sequencias_identicas(texto1, texto2)
        alteracoes = self.detectar_alteracoes(texto1, texto2)
        padrao = self.calcular_padrao_estrutural(texto1, texto2)
        
        if similaridade >= 0.8:
            nivel = "Alto"
        elif similaridade >= 0.5:
            nivel = "Médio"
        else:
            nivel = "Baixo"
            
        return AnaliseSintatica(
            similaridade=similaridade,
            padrao_estrutural=padrao,
            sequencias_identicas=sequencias,
            alteracoes_detectadas=alteracoes,
            nivel_similaridade=nivel
        )

    def realizar_analise_semantica(self, texto1: str, texto2: str) -> AnaliseSemantica:
        """
        Realiza análise semântica detalhada entre dois textos.
        """
        similaridade = self.calcular_similaridade_sequencia(texto1, texto2)
        trechos = self.encontrar_trechos_similares(texto1, texto2)
        palavras_chave = self.identificar_palavras_chave_comuns(texto1, texto2)
        contexto_similar = self.analisar_contexto(texto1, texto2)
        
        if similaridade >= 0.8:
            nivel = "Alto"
        elif similaridade >= 0.5:
            nivel = "Médio"
        else:
            nivel = "Baixo"
            
        return AnaliseSemantica(
            similaridade=similaridade,
            trechos_similares=trechos,
            palavras_chave_comuns=palavras_chave,
            contexto_similar=contexto_similar,
            nivel_similaridade=nivel
        )

    def verificar_plagio(self, texto1: str, texto2: str) -> PlagiarismReport:
        """
        Realiza uma análise completa de plágio com relatórios separados.
        """
        if not isinstance(texto1, str) or not isinstance(texto2, str):
            raise ValueError("Ambos os textos devem ser strings")
            
        if not texto1.strip() or not texto2.strip():
            raise ValueError("Os textos não podem estar vazios")
            
        texto1_prep = self.preprocessar_texto(texto1)
        texto2_prep = self.preprocessar_texto(texto2)
        
        analise_sintatica = self.realizar_analise_sintatica(texto1_prep, texto2_prep)
        analise_semantica = self.realizar_analise_semantica(texto1_prep, texto2_prep)
        
        plagio_detectado = (
            analise_sintatica.similaridade >= self.limiar_sintatico or
            analise_semantica.similaridade >= self.limiar_semantico
        )
        
        relatorio = PlagiarismReport(
            texto_original=texto1,
            texto_suspeito=texto2,
            analise_semantica=analise_semantica,
            analise_sintatica=analise_sintatica,
            plagio_detectado=plagio_detectado,
            data_analise=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            metadados={
                "tamanho_texto1": len(texto1.split()),
                "tamanho_texto2": len(texto2.split())
            }
        )
        
        self.historico_analises.append(relatorio)
        return relatorio


    def apresentar_resultados(self, relatorio: PlagiarismReport) -> None:
        """
        Apresenta os resultados detalhados da análise.
        """
        print("\n" + "="*60)
        print("RELATÓRIO DETALHADO DE ANÁLISE DE PLÁGIO")
        print("="*60)
        
        print("\nRESULTADO GERAL:")
        print(f"Data da análise: {relatorio.data_analise}")
        print(f"Plágio detectado: {'SIM' if relatorio.plagio_detectado else 'NÃO'}")
        
        print("\nANÁLISE SINTÁTICA:")
        print(f"Nível de Similaridade: {relatorio.analise_sintatica.nivel_similaridade}")
        print(f"Similaridade TF-IDF: {relatorio.analise_sintatica.similaridade:.2%}")
        print(f"Padrão Estrutural: {relatorio.analise_sintatica.padrao_estrutural:.2%}")
        
        if relatorio.analise_sintatica.sequencias_identicas:
            print("\nSequências Idênticas Encontradas:")
            for seq in relatorio.analise_sintatica.sequencias_identicas[:3]:
                print(f"- {seq}")
                
        if relatorio.analise_sintatica.alteracoes_detectadas:
            print("\nAlterações Detectadas:")
            for original, alterado in relatorio.analise_sintatica.alteracoes_detectadas[:3]:
                print(f"- {original} → {alterado}")
        
        print("\nANÁLISE SEMÂNTICA:")
        print(f"Nível de Similaridade: {relatorio.analise_semantica.nivel_similaridade}")
        print(f"Similaridade de Sequência: {relatorio.analise_semantica.similaridade:.2%}")
        print(f"Contexto Similar: {'Sim' if relatorio.analise_semantica.contexto_similar else 'Não'}")
        
        if relatorio.analise_semantica.palavras_chave_comuns:
            print("\nPalavras-chave Comuns:")
            print(", ".join(relatorio.analise_semantica.palavras_chave_comuns[:10]))
        
        if relatorio.analise_semantica.trechos_similares:
            print("\nTrechos Semanticamente Similares:")
            for i, (original, suspeito, similaridade) in enumerate(
                relatorio.analise_semantica.trechos_similares[:2], 1):
                print(f"\nPar #{i} (Similaridade: {similaridade:.2%})")
                print(f"Original : {original}")
                print(f"Suspeito: {suspeito}")
        
        print("\nMETADADOS:")
        print(f"Tamanho do texto original: {relatorio.metadados['tamanho_texto1']} palavras")
        print(f"Tamanho do texto suspeito: {relatorio.metadados['tamanho_texto2']} palavras")
        
        print("\n" + "="*60 + "\n")

def executar_exemplos():
    """
    Executa exemplos de uso do detector de plágio.
    """
    detector = EnhancedPlagioDetector(
        limiar_sintatico=0.6,
        limiar_semantico=0.6,
        tamanho_minimo_trecho=8
    )

    # Exemplo 1: Plágio direto com pequenas alterações
    texto_original_1 = """
    O aquecimento global é um fenômeno climático que indica o aumento da 
    temperatura média da Terra ao longo das últimas décadas. Este processo 
    está diretamente relacionado com a intensificação do efeito estufa, 
    causado principalmente pela emissão de gases poluentes na atmosfera.
    Os principais gases causadores do efeito estufa são o dióxido de 
    carbono, o metano e o óxido nitroso.
    """
    
    texto_suspeito_1 = """
    O aquecimento global é um fenômeno climático que mostra o aumento da 
    temperatura média do planeta Terra nas últimas décadas. Este processo 
    está diretamente ligado com a intensificação do efeito estufa, 
    causado principalmente pela liberação de gases poluentes na atmosfera.
    Os principais gases que causam o efeito estufa são o dióxido de 
    carbono, o metano e o óxido nitroso.
    """

    # Exemplo 2: Plágio com paráfrase
    texto_original_2 = """
    A Revolução Industrial, iniciada na Inglaterra no século XVIII,
    foi um período de grandes transformações tecnológicas e sociais.
    O processo de produção, antes artesanal e manual, passou a ser
    realizado por máquinas, aumentando significativamente a produtividade.
    Esta mudança teve profundo impacto na organização do trabalho e
    na vida dos trabalhadores.
    """
    
    texto_suspeito_2 = """
    As transformações tecnológicas e sociais marcaram o período conhecido
    como Revolução Industrial, que começou na Inglaterra durante o século XVIII.
    A forma de produção se transformou completamente, substituindo o trabalho
    manual e artesanal pela produção mecanizada, o que levou a um grande
    aumento na produtividade. Essas alterações mudaram drasticamente como
    o trabalho era organizado e afetaram profundamente a vida dos operários.
    """

    # Exemplo 3: Textos diferentes sobre o mesmo tema
    texto_original_3 = """
    A fotossíntese é um processo fundamental para a vida na Terra.
    Durante este processo, as plantas utilizam a luz solar, água e
    dióxido de carbono para produzir glicose e oxigênio. A clorofila,
    pigmento verde presente nas folhas, é essencial para captar a
    energia solar necessária para realizar este processo.
    """
    
    texto_suspeito_3 = """
    As plantas são organismos autótrofos que produzem seu próprio alimento
    através da fotossíntese. Este processo bioquímico ocorre nas células
    vegetais e requer luz solar, que é absorvida pela clorofila. Os
    produtos finais são glicose, que serve como fonte de energia para
    a planta, e oxigênio, que é liberado na atmosfera.
    """

    exemplos = [
        (texto_original_1, texto_suspeito_1, "EXEMPLO 1: PLÁGIO DIRETO"),
        (texto_original_2, texto_suspeito_2, "EXEMPLO 2: PLÁGIO COM PARÁFRASE"),
        (texto_original_3, texto_suspeito_3, "EXEMPLO 3: TEXTOS RELACIONADOS")
    ]

    for texto_original, texto_suspeito, titulo in exemplos:
        print(f"\n{'='*70}")
        print(titulo)
        print('='*70)
        
        relatorio = detector.verificar_plagio(texto_original, texto_suspeito)
        detector.apresentar_resultados(relatorio)

if __name__ == "__main__":
    try:
        executar_exemplos()
    except Exception as e:
        print(f"Erro ao executar os exemplos: {str(e)}")


EXEMPLO 1: PLÁGIO DIRETO
Erro ao calcular similaridade TF-IDF: The 'stop_words' parameter of TfidfVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'estes', 'deles', 'foi', 'estivéramos', 'seus', 'tenha', 'por', 'entre', 'estiveram', 'houvemos', 'tinha', 'ela', 'houveria', 'haver', 'e', 'seria', 'fôramos', 'tem', 'seremos', 'estivemos', 'forem', 'mas', 'esteve', 'as', 'estas', 'tenham', 'houveríamos', 'estivesse', 'muito', 'estivéssemos', 'me', 'ou', 'o', 'terão', 'sem', 'se', 'está', 'não', 'estávamos', 'houverem', 'para', 'havemos', 'mesmo', 'quando', 'somos', 'houveremos', 'houvermos', 'tém', 'tivemos', 'teríamos', 'lhes', 'aquela', 'estiver', 'nós', 'que', 'fomos', 'hão', 'tinham', 'estar', 'uma', 'tiver', 'aos', 'nossos', 'nos', 'só', 'esteja', 'teriam', 'tivéramos', 'isto', 'estão', 'haja', 'houvéssemos', 'essa', 'tivessem', 'seriam', 'te', 'num', 'hajamos', 'temos', 'às', 'era', 'houveram', 'estivessem', 'você', 'foram', 'essas', 'aqueles', 'for', 

In [4]:
# import nltk
# nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# !pip install sentence-transformers nltk tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.4 MB/s eta 0:00:00a 0:00:01
